# HPC 6.2 Automating the workflow on ARC

We are now going to transfer this workflow over to ARC, automating parts of it as we go.

The first step is to connect to ARC via `ssh`, as we are off campus we need to do this via `remote-access.leeds.ac.uk` first:

`ssh <username>@remote-access.leeds.ac.uk`    
`ssh <username>@arc4.leeds.ac.uk`

We recommend to follow the instructions in our KB article to make this process a bit easier.


We are going to use Anaconda to download and install our tools.  First:

`module load anaconda`

`conda config --add channels bioconda`

Next, we are going to create an **Anaconda Environment** just for our experiment but in order to do this we need to put a list of the tools we need inside a file called `requirements.txt`

In [ ]:
%%writefile requirements.txt
fastqc
trimmomatic
bwa
samtools=1.12
bcftools

and once we have that we need to tell Anaconda to make the environment based on what's in the file.

`conda env create --name varcall --file requirements.txt`

Which might take a few minutes.

Once it has completed we can check if that environment has been created:

`conda env list`

and change over in to it:

`source activate varcall`

Just like before, we can check if the tools work at the command line:

`bwa`

The final stage of the initial setup is to create a directory on `/nobackup` for our project work:

`mkdir -p /nobackup/$USER/vc_pipeline/data/untrimmed_fastq/`

This will give us a project directory (`vc_pipeline`) our data directory and a directory `untrimmed_fastq/` within that for our source data.

## Part 1: Downloading data

Our first HPC job is to download the data and put it inside this directory we just created.  We could do this manually but it's more useful to do this with a `batch job`.

We have a special `data` queue to do this so we don't clog up the regular queues with non-computational jobs.

In [ ]:
%%writefile 1_download_data.sh
# Submission script for download job
# Martin Callaghan yyyy-mm-dd

# Run from the current directory and with current environment
#$ -cwd -V

# Ask for some time (hh:mm:ss max of 48:00:00)
#$ -l h_rt=01:00:00

# Run in the data queue
#$ -P data

# Ask for some memory (by default, 1G, without a request)
#$ -l h_vmem=2G

# Send emails when job starts and ends
#$ -m be

# Now run the job
cd /nobackup/$USER/vc_pipeline/data/untrimmed_fastq/

curl -O ftp://ftp.sra.ebi.ac.uk/vol1/fastq/SRR258/004/SRR2589044/SRR2589044_1.fastq.gz
curl -O ftp://ftp.sra.ebi.ac.uk/vol1/fastq/SRR258/004/SRR2589044/SRR2589044_2.fastq.gz
curl -O ftp://ftp.sra.ebi.ac.uk/vol1/fastq/SRR258/003/SRR2584863/SRR2584863_1.fastq.gz
curl -O ftp://ftp.sra.ebi.ac.uk/vol1/fastq/SRR258/003/SRR2584863/SRR2584863_2.fastq.gz
curl -O ftp://ftp.sra.ebi.ac.uk/vol1/fastq/SRR258/006/SRR2584866/SRR2584866_1.fastq.gz
curl -O ftp://ftp.sra.ebi.ac.uk/vol1/fastq/SRR258/006/SRR2584866/SRR2584866_2.fastq.gz 

We can now send this job to the queue to be executed by the **batch scheduler**:

`qsub download_data.sh`

and you can check the progress of the job with the command:

`qstat`

When it's completed, you'll be able to go and see if the files have been downloaded:

`ls -l /nobackup/$USER/vc_pipeline/data/untrimmed_fastq/`

At the momnt, the script we wrote is just in our home directory.  We should put it inside our project directory in a new `scripts` subdirectory.

`mkdir /nobackup/$USER/vc_pipeline/scripts/`  
`mv download_data.sh /nobackup/$USER/vc_pipeline/scripts/`

## Part 2: Analysing quality with fastqc



Let's create a script using nano to run the `fastqc` part of the pipeline:

`nano /nobackup/$USER/vc_pipeline/scripts/read_qc.sh`

`fastqc` can use multiple cores so we can amend our submission script accordingly:

In [ ]:
%%writefile 2_read_qc.sh
# Submission script for fastqc job
# Martin Callaghan yyyy-mm-dd

# Run from the current directory and with current environment
#$ -cwd -V

# Ask for some time (hh:mm:ss max of 48:00:00)
#$ -l h_rt=01:00:00

# Ask for some memory (by default, 1G, without a request)
#$ -l h_vmem=4G

# Request 4 cores
#$ -pe smp 4

# Send emails when job starts and ends
#$ -m be

# Now run the job
module load anaconda
source activate varcall

cd /nobackup/$USER/vc_pipeline/data/untrimmed_fastq/

echo "Running FastQC ..."
fastqc *.fastq*

mkdir -p /nobackup/$USER/vc_pipeline/results/fastqc_untrimmed_reads

echo "Saving FastQC results..."
mv *.zip /nobackup/$USER/vc_pipeline/results/fastqc_untrimmed_reads/
mv *.html /nobackup/$USER/vc_pipeline/results/fastqc_untrimmed_reads/

cd /nobackup/$USER/vc_pipeline/results/fastqc_untrimmed_reads/

echo "Unzipping..."
for filename in *.zip
    do
    unzip $filename
    done

echo "Saving summary..."
mkdir /nobackup/$USER/vc_pipeline/docs
cat */summary.txt > /nobackup/$USER/vc_pipeline/docs/fastqc_summaries.txt



Now we can submit the job and wait for it to be completed:

`qsub /nobackup/$USER/vc_pipeline/scripts/read_qc.sh`

Remembering that `qstat` will show us the progress of the job.

**NOTE**: At this stage in real life you would need to do some assessment of the quality of your reads to determine the parameters of the `trimmomatic` command.

## Part 3: Trimming

Just like in our previous Notebook, we now need to call `trimmomatic` on all of our **untrimmed** data to create the **trimmed** data for the next step.

Create this file using nano:

`nano /nobackup/issmcal/vc_pipeline/scripts/run_trims.sh`

In [ ]:
%%writefile 3_run_trims.sh
# Submission script for trimming job
# Martin Callaghan yyyy-mm-dd

# Run from the current directory and with current environment
#$ -cwd -V

# Ask for some time (hh:mm:ss max of 48:00:00)
#$ -l h_rt=02:00:00

# Ask for some memory (by default, 1G, without a request)
#$ -l h_vmem=4G

# Request 4 cores
#$ -pe smp 4

# Send emails when job starts and ends
#$ -m be

# Now run the job
# Change to the correct directory
cd /nobackup/$USER/vc_pipeline/data/untrimmed_fastq

# Download the Nextera adapter
! wget https://raw.githubusercontent.com/timflutre/trimmomatic/master/adapters/NexteraPE-PE.fa

# Loop over the untrimmed fastq files

for infile in *_1.fastq.gz
do
   base=$(basename ${infile} _1.fastq.gz)
   trimmomatic PE ${infile} ${base}_2.fastq.gz \
                ${base}_1.trim.fastq.gz ${base}_1un.trim.fastq.gz \
                ${base}_2.trim.fastq.gz ${base}_2un.trim.fastq.gz \
                SLIDINGWINDOW:4:20 MINLEN:25 ILLUMINACLIP:NexteraPE-PE.fa:2:40:15 
done

# move our trimmed FASTQ files to a new subdirectory within our data/ directory
mkdir -p /nobackup/$USER/vc_pipeline/data/trimmed_fastq
mv *.trim* /nobackup/$USER/vc_pipeline/data/trimmed_fastq

## Part 4: Automating the rest of the Variant Calling workflow

We can extend these principles to the entire variant calling workflow. To do this, we will take all of the individual commands that we wrote before, put them into a single file, add variables so that the script knows to iterate through our input files and write to the appropriate output files. 

This is very similar to what we did with our `read_qc.sh` script, but will be a bit more complex.

Again, using `nano`:

`nano /nobackup/issmcal/vc_pipeline/scripts/vc_script.sh`

Submit to the queue with:

`qsub /nobackup/issmcal/vc_pipeline/scripts/vc_script.sh`

And watch progress with `qstat`

In [ ]:
%%writefile 4_vc_script.sh
# Submission script for vc workflow job
# Martin Callaghan yyyy-mm-dd

# Run from the current directory and with current environment
#$ -cwd -V

# Ask for some time (hh:mm:ss max of 48:00:00)
#$ -l h_rt=02:00:00

# Ask for some memory (by default, 1G, without a request)
#$ -l h_vmem=4G

# Request 4 cores
#$ -pe smp 4

# Send emails when job starts and ends
#$ -m be

# Now run the job
module load anaconda
source activate varcall

# Download the reference genome
mkdir -p /nobackup/$USER/vc_pipeline/data/ref_genome
curl -L -o /nobackup/$USER/vc_pipeline/data/ref_genome/ecoli_rel606.fasta.gz ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/000/017/985/GCA_000017985.1_ASM1798v1/GCA_000017985.1_ASM1798v1_genomic.fna.gz
gunzip /nobackup/$USER/vc_pipeline/data/ref_genome/ecoli_rel606.fasta.gz

cd /nobackup/$USER/vc_pipeline/results

genome=/nobackup/$USER/vc_pipeline/data/ref_genome/ecoli_rel606.fasta

bwa index $genome

mkdir -p sam bam bcf vcf

for fq1 in /nobackup/$USER/vc_pipeline/data/trimmed_fastq/*_1.trim.fastq*
    do
    echo "working with file $fq1"

    base=$(basename $fq1 _1.trim.fastq)
    echo "base name is $base"

    fq1=/nobackup/$USER/vc_pipeline/data/trimmed_fastq/${base}_1.trim.fastq
    fq2=/nobackup/$USER/vc_pipeline/data/trimmed_fastq/${base}_2.trim.fastq

    sam=/nobackup/$USER/vc_pipeline/results/sam/${base}.aligned.sam
    bam=/nobackup/$USER/vc_pipeline/results/bam/${base}.aligned.bam

    sorted_bam=/nobackup/$USER/vc_pipeline/results/bam/${base}.aligned.sorted.bam
    raw_bcf=/nobackup/$USER/vc_pipeline/results/bcf/${base}_raw.bcf

    variants=/nobackup/$USER/vc_pipeline/results/bcf/${base}_variants.vcf
    final_variants=/nobackup/$USER/vc_pipeline/results/vcf/${base}_final_variants.vcf 

    bwa mem $genome $fq1 $fq2 > $sam

    samtools view -S -b $sam > $bam
    samtools sort -o $sorted_bam $bam 

    samtools index $sorted_bam

    bcftools mpileup -O b -o $raw_bcf -f $genome $sorted_bam
    bcftools call --ploidy 1 -m -v -o $variants $raw_bcf 
    vcfutils.pl varFilter $variants > $final_variants
   
    done

Writing 4_vc_script.sh
